# Linear Regession

Derived from:

> https://stackabuse.com/linear-regression-in-python-with-scikit-learn/
> https://docs.scipy.org/doc/numpy/reference/generated/numpy.polyfit.html?highlight=polyfit#numpy.polyfit

## Generate Some Time-Based Data & Predict

Idea is to:

- Generate some sample data looking like its' time-based
- Perform a linear regression
- Predict 30s ahead

In [1]:
#
# set up standard imports in point bokeh to the notebook
#
import pandas as pd
import numpy as np
import time
import random
import util
from tqdm import tqdm
from bokeh.plotting import figure, show, gridplot
from bokeh.io import output_notebook

output_notebook()

Loading BokehJS ...

## Generate Some Data

In [2]:
# Number of days to generate data for
DAYS = 1

# generate a numpy array of raw data first
d = util.gen_data(days=DAYS)

# turn it into a pandas data frame
df = pd.DataFrame({'Time': d[:, 0], 'ACEs': d[:, 1]})

## Plot Data

In [3]:
p = figure(plot_width=600)
p.line(x=df['Time'], y=df['ACEs'], color='steelblue')

show(p)

## Linear Regression

Let's do linear regression using two libraries. First, SciKit Learn, then numpy.

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

# split data in the x and y axes
x = df.iloc[:, :-1].values
y = df.iloc[:, 1].values

# after all that, run the LR algorithm
regressor = LinearRegression()
regressor.fit(x, y)

# ...and display the resulting intercept and co-efficient
print('Intercept : %0.9f' % regressor.intercept_)
print('Slope     : %0.9f' % regressor.coef_[0])

Intercept : 193.052430797
Slope     : 0.097374237


In [5]:
# split data in the x and y axes
x = df['Time'].values
y = df['ACEs'].values

# use numpy to calculate linear regression
fit = np.polyfit(x, y, 1, full=True)

print('Intercept : %0.9f' % fit[0][1])
print('Slope     : %0.9f' % fit[0][0])

Intercept : 193.052430797
Slope     : 0.097374237


## Plot Real Data & LR Curve

We can use the intercept and coefficient from either algorithm, but we will see that taken over an extended period it's a pretty bad fit!

In [6]:
# generate the time axis
t = np.arange(0, 1440)

intercept = fit[0][1]
c = fit[0][0]

# linear regression curve
y_lr = [intercept + c * x for x in t]

# plot
p = figure(plot_width=900, plot_height=600)
p.line(x=t, y=y, color='green', legend_label='Observed Data')
p.line(x=t, y=y_lr, color='red', legend_label='LR Curve')
show(p)

## Can We Still Use LR?

Maybe. What if instead of looking at the entire data set, we instead consider a prediction use case, where we want to see if the **current** LR curve predicts that we will breach some arbitrary threshold within a certain time period.

But, as we will see, we need to be cafeful!!

In [10]:
# where to start in data set to ignore boring data
START = 400

# how many samples to perform the LR over
WINDOW_SIZE  = 20

# how many samples to perform the LR over
DISPLAY_SIZE  = 30

# how often to run the LR
SAMPLE_EVERY = 5

# how far ahead do we want to try and predict?
LOOKAHEAD    = 10

# what is the arbitrary limit?
BREACH       = 400

# generate a numpy array of raw data first
d = util.gen_data(days=DAYS)
x = d[:, 0]
y = d[:, 1]

plot_list = []
for i in tqdm(x[START:]):
    
    # only do something if we have enough samples AND we're at a sample point
    if (i >= WINDOW_SIZE) and not (i % SAMPLE_EVERY):

        # take the window of data to do LR over
        x_display = x[i-DISPLAY_SIZE:i]
        y_display = y[i-DISPLAY_SIZE:i]
        
        # take the window of data to do LR over
        x_lr = x[i-WINDOW_SIZE:i]
        y_lr = y[i-WINDOW_SIZE:i]
        
        # do the linear regression using numpy
        fit = np.polyfit(x_lr, y_lr, 1, full=True)
        intercept = fit[0][1]
        c = fit[0][0]

        # plot the sample data if the gradient > 0
        if c > 0.0:

            # title of plot, the equation
            title = 'time = %d' % i
            
            # scatter plot of raw data plus LR curve
            p = figure(plot_width=300, plot_height=300, title=title, toolbar_location=None)
            
            # mark the display points of real data so far
            p.circle(x=x_display, y=y_display, color='navy', size=3)
            
            # overlay the LR-calculated curve, up to the prediction
            x_lr = np.append(x_lr, x[i + LOOKAHEAD])
            p.line(x=x_lr, y=[intercept + c * x for x in x_lr], color='red', line_width=2)
            
            # add the constant threshold line
            x_display = np.append(x_display, i + LOOKAHEAD)
            y_thresh = [BREACH for _ in x_display]
            p.line(x=x_display, y=y_thresh, color='orange', line_width=2)
            
            # mark the lookahead prediction
            p.x(x=[i+LOOKAHEAD], y=[intercept + c * (i+LOOKAHEAD)], color='red', size=10)
            
            # Mark the real values between now and the prediction
            p.x(x=x[i:i+LOOKAHEAD], y=y[i:i+LOOKAHEAD], color='darkgreen', size=6)
            
            # add plot to plot list
            plot_list.append(p)
            
            # check if calculate we've breached the threshhold at now + LOOKAHEAD
            check_prediction = intercept + c * (i + LOOKAHEAD)
            check_real = y[i + LOOKAHEAD]
            if (check_prediction > BREACH) and (check_real > BREACH):
                print('Hit exit condition at time t=%d!' % i)
                break
            if (check_prediction > BREACH) or (check_real > BREACH):
                SAMPLE_EVERY = 1

                
# reshape plot list


modded = len(plot_list) % 3
if modded > 0:
    missing = 3 - modded
    for _ in (0, missing): plot_list.append(None)
        
plot_list = util.reshape(plot_list, [2, 3])
p = gridplot(plot_list)
show(p)

 14%|█▍        | 146/1040 [00:00<00:02, 411.96it/s]


Hit exit condition at time t=558!
